In [1]:
import pyspark
from pyspark.context import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number, expr, explode, udf, col,  monotonically_increasing_id, when
from pyspark.sql.types import StructType, IntegerType, StringType, ByteType
from pyspark.sql.window import Window
import datetime, time
import random

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1672841601030_0001,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Data Preparation

In [2]:
# Read campaign experiment data
h1 = spark.read \
    .parquet('s3://panel-ml-incrementality-data/incrementality/flipper-experiment-data/3bc859c4-5398-4eb1-89d5-17466537a8bf/*/*')
h1.createOrReplaceTempView("h1_v")

h2 = spark.read \
    .parquet('s3://panel-ml-incrementality-data/incrementality/flipper-experiment-data/22d45946-f091-4be3-92e5-c45ed375de30/*/*')
h2.createOrReplaceTempView("h2_v")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Union camapign experiment data
full_data = spark.sql("""
select *,
'Halloween1' as experiment 
from h1_v 
where audienceFunnel = 'FLIPPER_AD_EXPOSURE_AUDIENCE'
union
select adUserId, audienceFunnel, recordedTreatmentStatus, date,'Halloween2' as experiment  
from h2_v
where audienceFunnel = 'FLIPPER_EXPERIMENT_AUDIENCE'
""")
full_data.createOrReplaceTempView("df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Check camapign date
spark.sql("""select min(date), max(date) from df""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+
| min(date)| max(date)|
+----------+----------+
|2022-08-15|2022-10-31|
+----------+----------+

In [25]:
# today date
today = spark.sql("""select date_format(current_date(), "yyyyMMDD")""").first()[0]
today

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'20230104'

In [10]:
experiment = 'halloween'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Hershey's brands
brandaidids = """
('BRAND|BRAND_AID|188fab47-dd2f-3ca6-ae64-efc94d762157',
         'BRAND|BRAND_AID|1de8f676-e136-3a10-a9b1-bbb62445b067',
         'BRAND|BRAND_AID|2bbc78e8-d532-39ad-b0e0-9f94dd0ccf9d',
         'BRAND|BRAND_AID|44af4ccd-f346-38de-9712-6986c0c55604',
         'BRAND|BRAND_AID|4acc765e-10a9-37ab-a5db-b71b60e676e7',
         'BRAND|BRAND_AID|4dd231fa-7959-3816-97ff-f4870c165edb',
         'BRAND|BRAND_AID|4e8b140d-b7a5-31f9-aa55-ca700f9391e9',
         'BRAND|BRAND_AID|6e2e645c-6641-3c1a-8a4e-384fbd1654ef',
         'BRAND|BRAND_AID|7c28d380-6e2a-3b57-9c5e-ad1113a56d47',
         'BRAND|BRAND_AID|7cc191d1-6226-370b-94b0-e97cf44cf33d',
         'BRAND|BRAND_AID|8ccd4650-9a1d-3244-a7cd-dac899b81cab',
         'BRAND|BRAND_AID|c0193602-6a44-391d-a65c-87183d5a695b',
         'BRAND|BRAND_AID|cda0ae9f-a145-3110-9a3b-c4d1c064bcb2',
         'BRAND|BRAND_AID|d23eb567-2d82-3eea-af2b-6766f995788b',
         'BRAND|BRAND_AID|d6a1e8bd-c39b-3984-be75-31003d4aa49e',
         'BRAND|BRAND_AID|de6e7b73-4030-3978-85a5-b084634f27f4',
         'BRAND|BRAND_AID|e0680ca5-4ff5-3b4d-99a5-6aabd8c4a932',
         'BRAND|BRAND_AID|fdbb33c1-602c-3c7b-94b1-3fca11816c0a',
         'BRAND|BRAND_AID|2bbc78e8-d532-39ad-b0e0-9f94dd0ccf9d',
         'BRAND|BRAND_AID|44af4ccd-f346-38de-9712-6986c0c55604',
         'BRAND|BRAND_AID|4acc765e-10a9-37ab-a5db-b71b60e676e7',
         'BRAND|BRAND_AID|4dd231fa-7959-3816-97ff-f4870c165edb',
         'BRAND|BRAND_AID|7cc191d1-6226-370b-94b0-e97cf44cf33d',
         'BRAND|BRAND_AID|d6a1e8bd-c39b-3984-be75-31003d4aa49e',
         'BRAND|BRAND_AID|e0680ca5-4ff5-3b4d-99a5-6aabd8c4a932')
"""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Create experiment table for future use
exposure = spark.sql("""
    select audienceFunnel, recordedTreatmentStatus, adUserId, date, experiment,
        min(date)over() as exp_start_date, max(date)over() as exp_end_date,
        (datediff(max(date)over(), min(date)over()) + 1) as ads_exp_length
    from df 
    group by 1,2,3,4,5
    CLUSTER BY adUserId, date""")

exposure.createOrReplaceTempView("exposure")

exposure.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------------+--------------------+----------+----------+--------------+------------+--------------+
|      audienceFunnel|recordedTreatmentStatus|            adUserId|      date|experiment|exp_start_date|exp_end_date|ads_exp_length|
+--------------------+-----------------------+--------------------+----------+----------+--------------+------------+--------------+
|FLIPPER_AD_EXPOSU...|                     V3|010100005e1968887...|2022-08-15|Halloween1|    2022-08-15|  2022-10-31|            78|
|FLIPPER_AD_EXPOSU...|                      T|01010001d6dbe7b23...|2022-08-27|Halloween1|    2022-08-15|  2022-10-31|            78|
|FLIPPER_EXPERIMEN...|                      T|010100022d242307e...|2022-10-21|Halloween2|    2022-08-15|  2022-10-31|            78|
|FLIPPER_EXPERIMEN...|                     V3|01010002c3abeb152...|2022-10-29|Halloween2|    2022-08-15|  2022-10-31|            78|
|FLIPPER_AD_EXPOSU...|                      T|010100030d404de20...|20

In [7]:
# Create shopper panel table of Hershey's brand for future use
shopper_panel = spark.sql("""
    select 
        date(transaction_date) as transaction_date,
        p.purchase_date_impute as transaction_date_imputed,
        customer_id,
        t.receipt_id,
        t.transaction_id,
        brandaid_id,
        ubi_brand_name,
        item_price,
        p.raw_item_price_brand_5iqr_median_impute as item_price_imputed,    
        retailer_name
    from spektr_shopperpanel.panel_receipts_v2 r
         join spektr_shopperpanel.panel_transactions_v3 t
         on r.receipt_id = t.receipt_id
    left join spektr_shopperpanel.panel_transactions_quality_score_impute_price_trial p
         on p.line_item_id = t.transaction_id
    where status = 'APPROVED'
        and r.marketplace_id = 'ATVPDKIKX0DER'
        and customer_id <> 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        and customer_id is not null
        and brandaid_id in {0}
    CLUSTER BY  customer_id, transaction_date_imputed
    """.format(brandaidids))

shopper_panel.createOrReplaceTempView("sp")

shopper_panel.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+------------------+-------------+
|transaction_date|transaction_date_imputed|         customer_id|          receipt_id|      transaction_id|         brandaid_id|ubi_brand_name|          item_price|item_price_imputed|retailer_name|
+----------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+------------------+-------------+
|      2022-12-12|              2022-12-12|0101000260b1049d6...|010150c7bdf53e5dc...|0101458408c5edd65...|BRAND|BRAND_AID|4...|          null|2.000000000000000000|               2.0|      Walmart|
|      2022-12-12|              2022-12-12|0101000260b1049d6...|010150c7bdf53e5dc...|0101458408c5edd65...|BRAND|BRAND_AID|4...|          null|2.000000000000000000|               2.0|      Walmart|
|            nu

In [8]:
# Create shopper panel table of all brands for future use (matching)
shopper_panel_total = spark.sql("""
    select 
        date(transaction_date) as transaction_date,
        p.purchase_date_impute as transaction_date_imputed,
        customer_id,
        t.receipt_id,
        t.transaction_id,
        brandaid_id,
        ubi_brand_name,
        item_price,
        p.raw_item_price_brand_5iqr_median_impute as item_price_imputed,    
        retailer_name
    from spektr_shopperpanel.panel_receipts_v2 r
         join spektr_shopperpanel.panel_transactions_v3 t
         on r.receipt_id = t.receipt_id
    left join spektr_shopperpanel.panel_transactions_quality_score_impute_price_trial p
         on p.line_item_id = t.transaction_id
    where status = 'APPROVED'
        and r.marketplace_id = 'ATVPDKIKX0DER'
        and customer_id <> 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        and customer_id is not null
    CLUSTER BY  customer_id, transaction_date_imputed
    """)

shopper_panel_total.createOrReplaceTempView("sp_total")

shopper_panel_total.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+----------------+
|transaction_date|transaction_date_imputed|         customer_id|          receipt_id|      transaction_id|         brandaid_id|      ubi_brand_name|          item_price|item_price_imputed|   retailer_name|
+----------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+----------------+
|      2022-10-02|              2022-10-02|010100003f31b563e...|010180e52abad7570...|010115ec0f95611f7...|BRAND|BRAND_AID|e...|  Kirkland Signature|16.98999999999999...|             16.99|Costco Wholesale|
|      2022-10-02|              2022-10-02|010100003f31b563e...|010180e52abad7570...|010115ec0f95611f7...|BRAND|BRAND_AID|e...|  Kirkland Signature|16.98999999999999...|       

# Scaling Factor

In [11]:
# Scaling factor
sf_results = spark.sql("""
with exposure as (
    select date, audienceFunnel, recordedTreatmentStatus, adUserId, experiment
    from df 
    group by 1,2,3,4,5)
    
,  panelist as (
    select 
        distinct customer_id
    from spektr_shopperpanel.panel_receipts_v2 r
    join spektr_shopperpanel.panel_transactions_v3 t
        on r.receipt_id = t.receipt_id
    where status = 'APPROVED'
        and r.marketplace_id = 'ATVPDKIKX0DER'
        and customer_id <> 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        and customer_id is not null)

, match as (
    select 
        date, 
        experiment,
        recordedTreatmentStatus,
        count(distinct adUserId) as count_users,
        count(distinct panelist.customer_id) as count_mapped
    from exposure 
    left join panelist on exposure.adUserId = panelist.customer_id
    group by 1,2,3
    order by date, recordedTreatmentStatus)

select 
    experiment,
    recordedTreatmentStatus,
    sum(count_mapped)/sum(count_users) as matching_rate,
    sum(count_users)/sum(count_mapped) as scaling_factor
from match
group by 1,2

""")

sf_results.show()

sf_results.createOrReplaceTempView("sf_results")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------------+--------------------+------------------+
|experiment|recordedTreatmentStatus|       matching_rate|    scaling_factor|
+----------+-----------------------+--------------------+------------------+
|Halloween1|                     V3| 0.00395353018439633|252.93850137954402|
|Halloween1|                      T|0.003949988043811757|253.16532326386368|
|Halloween2|                      T|0.004879254599147...|204.94933799411018|
|Halloween2|                     V3|0.004916337084066291|203.40346540536686|
+----------+-----------------------+--------------------+------------------+

# Direct Scaling Method

# TTE

In [39]:
## If calculate in Excel

# attr_1 = spark.sql("""
# with exposure as 
#     (select audienceFunnel, recordedTreatmentStatus, adUserId, date, experiment,
#     min(date) over() as exp_start_date, max(date) over() as exp_end_date
#     from df 
#     where experiment='Halloween1'
#     group by 1,2,3,4,5)

# ,  sp as 
#     (select 
#     date(transaction_date) as transaction_date,
#     p.purchase_date_impute as transaction_date_imputed,
#     customer_id,
#     t.receipt_id,
#     t.transaction_id,
#     brandaid_id,
#     ubi_brand_name,
#     item_price,
#     p.raw_item_price_brand_5iqr_median_impute as item_price_imputed,    
#     retailer_name
#     from spektr_shopperpanel.panel_receipts_v2 r
#          join spektr_shopperpanel.panel_transactions_v3 t
#          on r.receipt_id = t.receipt_id
#     left join spektr_shopperpanel.panel_transactions_quality_score_impute_price_trial p
#          on p.line_item_id = t.transaction_id
#     where status = 'APPROVED'
#     and r.marketplace_id = 'ATVPDKIKX0DER'
#     and customer_id <> 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
#     and customer_id is not null
#     and brandaid_id in {0} 
#     and p.purchase_date_impute >= '2022-08-01'
#     )
    
# select
# transaction_date,
# transaction_date_imputed,
# recordedTreatmentStatus,
# retailer_name,
# customer_id,
# receipt_id,
# transaction_id,
# item_price,
# item_price_imputed,
# max(date) as ads_exp_date
# from exposure join sp on exposure.adUserId = sp.customer_id
# -- total window method
# and transaction_date_imputed between exp_start_date and date_add(exp_end_date, 14)
# -- remove transactions before each exposure
# and transaction_date_imputed >=date
# group by 1,2,3,4,5,6,7,8,9""".format(brandaidids))

# attr_1.repartition(1) \
# .write.mode('overwrite').csv("s3a://panel-ml-incrementality-data/incrementality/results_1205/attr_halloween_1",header=True)

# attr_1.createOrReplaceTempView('attr_1_v')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# attr_2 = spark.sql("""
# with exposure as 
#     (select audienceFunnel, recordedTreatmentStatus, adUserId, date, experiment,
#     min(date) over() as exp_start_date, max(date) over() as exp_end_date
#     from df 
#     where experiment='Halloween2'
#     group by 1,2,3,4,5)

# ,  sp as 
#     (select 
#     date(transaction_date) as transaction_date,
#     p.purchase_date_impute as transaction_date_imputed,
#     customer_id,
#     t.receipt_id,
#     t.transaction_id,
#     brandaid_id,
#     ubi_brand_name,
#     item_price,
#     p.raw_item_price_brand_5iqr_median_impute as item_price_imputed,    
#     retailer_name
#     from spektr_shopperpanel.panel_receipts_v2 r
#          join spektr_shopperpanel.panel_transactions_v3 t
#          on r.receipt_id = t.receipt_id
#     left join spektr_shopperpanel.panel_transactions_quality_score_impute_price_trial p
#          on p.line_item_id = t.transaction_id
#     where status = 'APPROVED'
#     and r.marketplace_id = 'ATVPDKIKX0DER'
#     and customer_id <> 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
#     and customer_id is not null
#     and brandaid_id in {0} 
#     and p.purchase_date_impute >= '2022-08-01'
#     )
    
# select
# transaction_date,
# transaction_date_imputed,
# recordedTreatmentStatus,
# retailer_name,
# customer_id,
# receipt_id,
# transaction_id,
# item_price,
# item_price_imputed,
# max(date) as ads_exp_date
# from exposure join sp on exposure.adUserId = sp.customer_id
# -- total window method
# and transaction_date_imputed between exp_start_date and date_add(exp_end_date, 14)
# -- remove transactions before each exposure
# and transaction_date_imputed >=date
# group by 1,2,3,4,5,6,7,8,9""".format(brandaidids))

# attr_2.repartition(1) \
# .write.mode('overwrite').csv("s3a://panel-ml-incrementality-data/incrementality/results_1205/attr_halloween_2",header=True)

# attr_2.createOrReplaceTempView('attr_2_v')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
sales_post_details = spark.sql("""
select 
    recordedTreatmentStatus,
    customer_id,
    adUserId,
    item_price_imputed,
    scaling_factor,
    scaled_item_price_imputed
from (
    select 
        exposure.recordedTreatmentStatus,
        adUserId,
        customer_id,
        sum(item_price_imputed) as item_price_imputed,
        scaling_factor,
        scaling_factor*sum(item_price_imputed) as scaled_item_price_imputed,
        max(date) as ads_exp_date 
    from exposure 
    left join sp on exposure.adUserId = sp.customer_id
        -- total window method
        and transaction_date_imputed between exp_start_date and date_add(exp_end_date, 14)
        -- remove transactions before each exposure
        and transaction_date_imputed >=date
    left join sf_results
        on exposure.experiment = sf_results.experiment
        and exposure.recordedTreatmentStatus = sf_results.recordedTreatmentStatus
    group by 1,2,3,scaling_factor
    )
""")

sales_post_details.show(100,False)
sales_post_details.createOrReplaceTempView("sales_post_details")

sales_post_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/sales_post_details/'
         , header=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+------------------+------------------+-------------------------+
|recordedTreatmentStatus|customer_id                                                         |adUserId                                                            |item_price_imputed|scaling_factor    |scaled_item_price_imputed|
+-----------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+------------------+------------------+-------------------------+
|T                      |0101015f0b7fd393e09a07ba6ae033c39bb1604827304c0190f56b1a1790134f99de|0101015f0b7fd393e09a07ba6ae033c39bb1604827304c0190f56b1a1790134f99de|3.0               |204.94933799411018|614.8480139823305        |
|V3                     |0101015f0b7fd393e09a07ba6ae033c39bb1604827304c0190f56b1a1790134

In [27]:
purchase_post_details = spark.sql("""
with joined as (
select
    transaction_date_imputed,
    recordedTreatmentStatus,
    exposure.adUserId,
    sp.customer_id,
    receipt_id,
    transaction_id,
    item_price_imputed,
    max(date) as ads_exp_date
from exposure 
left join sp on exposure.adUserId = sp.customer_id
    -- total window method
    and transaction_date_imputed between exp_start_date and date_add(exp_end_date, 14)
    -- remove transactions before each exposure
    and transaction_date_imputed >=date
group by 1,2,3,4,5,6,7
)

select 
    adUserId,
    recordedTreatmentStatus,
    case when customer_id is not null then 1 else 0 end as convert_flag
from joined
-- only take matched ads exposure and shopper panel users
where adUserId in (select distinct customer_id from sp_total)
group by 1,2,3
order by 1,2
""")

purchase_post_details.show(100,False)
purchase_post_details.createOrReplaceTempView("purchase_post_details")

purchase_post_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/purchase_post_details/'
         , header=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------------------+-----------------------+------------+
|adUserId                                                            |recordedTreatmentStatus|convert_flag|
+--------------------------------------------------------------------+-----------------------+------------+
|0101000080deb73785c55df2bd61249cfbc32c96bfd2e02042ca182cdfc4eaaee31c|T                      |0           |
|0101000218b2f35b4821119af19a44577a6c7ac684796e718577e0988186b16e14a0|T                      |0           |
|01010002d7da8481d9f112676cc0631de58b515247ad33038db8f1800aeb3558db43|V3                     |0           |
|010100033caa0e6df0fc9f65807c749bb7e5b245bf3d086a4d6e5111e78472c0f38c|T                      |0           |
|010100046c1d157eab523c8a612bc990a8b497478f1cbafc4f993014cd4a64b7c949|V3                     |0           |
|010100051ae331bcf65ec335aef240c761d964c9a271141470e74bd172423b098df6|V3                     |0           |
|0101000574bbd0c60e315594ddb

# DID

### DID Sales

In [28]:
# Pre period
sales_pre_details = spark.sql("""
select distinct
    recordedTreatmentStatus,
    adUserId,
    customer_id,
    item_price_imputed,
    scaling_factor,
    scaled_item_price_imputed
from (
    select 
        exposure.recordedTreatmentStatus,
        exposure.adUserId,
        sp.customer_id,
        sum(item_price_imputed) as item_price_imputed,
        scaling_factor,
        scaling_factor*sum(item_price_imputed) as scaled_item_price_imputed
    from exposure 
    left join sp on exposure.adUserId = sp.customer_id
        -- total window method
        and transaction_date_imputed >= date_sub(exp_start_date, 14 + ads_exp_length) 
        and transaction_date_imputed < exp_start_date
    left join sf_results
        on exposure.experiment = sf_results.experiment
        and exposure.recordedTreatmentStatus = sf_results.recordedTreatmentStatus
    group by 1,2,3,scaling_factor
)
""")


sales_pre_details.show(100,False)
sales_pre_details.createOrReplaceTempView("sales_pre_details")

sales_pre_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/sales_pre_details/'
         , header=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+------------------+------------------+-------------------------+
|recordedTreatmentStatus|adUserId                                                            |customer_id                                                         |item_price_imputed|scaling_factor    |scaled_item_price_imputed|
+-----------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+------------------+------------------+-------------------------+
|T                      |0101015f0b7fd393e09a07ba6ae033c39bb1604827304c0190f56b1a1790134f99de|0101015f0b7fd393e09a07ba6ae033c39bb1604827304c0190f56b1a1790134f99de|7.92              |204.94933799411018|1623.1987569133526       |
|V3                     |0101015f0b7fd393e09a07ba6ae033c39bb1604827304c0190f56b1a1790134

In [38]:
pre_col_list = [col(col_name).alias("pre_" + col_name)  for col_name in sales_pre_details.columns]
post_col_list = [col(col_name).alias("post_" + col_name)  for col_name in sales_post_details.columns]
sales_pre_details_new = sales_pre_details.select(*pre_col_list)
sales_pre_details_new.show()
sales_post_details_new = sales_post_details.select(*post_col_list)
sales_post_details_new.show()

sales_pre_details_new.createOrReplaceTempView("sales_pre_details_new")
sales_post_details_new.createOrReplaceTempView("sales_post_details_new")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+--------------------+--------------------+----------------------+------------------+-----------------------------+
|pre_recordedTreatmentStatus|        pre_adUserId|     pre_customer_id|pre_item_price_imputed|pre_scaling_factor|pre_scaled_item_price_imputed|
+---------------------------+--------------------+--------------------+----------------------+------------------+-----------------------------+
|                          T|0101015f0b7fd393e...|0101015f0b7fd393e...|                  7.92|204.94933799411018|           1623.1987569133526|
|                         V3|0101015f0b7fd393e...|0101015f0b7fd393e...|                  7.92|252.93850137954402|           2003.2729309259887|
|                          T|0101017bde19ae1ce...|0101017bde19ae1ce...|                  4.56|253.16532326386368|           1154.4338740832184|
|                          T|01010268ebcbd6aac...|01010268ebcbd6aac...|                  3.92|204.94933799411018|             803.401404

In [39]:
# diff of pre and post
diff_sales_details = spark.sql("""

with joined as(
select * 
from sales_pre_details_new pre
right join sales_post_details_new post
on pre.pre_adUserId = post.post_adUserId
and pre.pre_recordedTreatmentStatus = post.post_recordedTreatmentStatus
)

,joined_trans as(
select 
       pre_recordedTreatmentStatus, pre_adUserId, pre_customer_id,
       post_recordedTreatmentStatus, post_adUserId, post_customer_id,
       coalesce(pre_customer_id, post_customer_id) as customer_id,
       coalesce(pre_adUserId, post_adUserId) as adUserId,
       coalesce(pre_recordedTreatmentStatus, post_recordedTreatmentStatus) as recordedTreatmentStatus,
       sum(case when pre_item_price_imputed is null then 0 else pre_item_price_imputed end) as pre_item_price_imputed,
       sum(case when pre_scaled_item_price_imputed is null then 0 else pre_scaled_item_price_imputed end) as pre_scaled_item_price_imputed,
       sum(case when post_item_price_imputed is null then 0 else post_item_price_imputed end) as post_item_price_imputed,
       sum(case when post_scaled_item_price_imputed is null then 0 else post_scaled_item_price_imputed end) as post_scaled_item_price_imputed 
from joined
group by 1,2,3,4,5,6,7,8,9
) 

select distinct *,
       post_item_price_imputed - pre_item_price_imputed as diff_item_price,
       post_scaled_item_price_imputed - pre_scaled_item_price_imputed as diff_scaled_item_price
from joined_trans
""")

diff_sales_details.show()

diff_sales_details.createOrReplaceTempView("diff_sales_details")
diff_sales_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/diff_sales_details/'
         , header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+----------------------+-----------------------------+-----------------------+------------------------------+--------------------+----------------------+
|pre_recordedTreatmentStatus|        pre_adUserId|     pre_customer_id|post_recordedTreatmentStatus|       post_adUserId|    post_customer_id|         customer_id|            adUserId|recordedTreatmentStatus|pre_item_price_imputed|pre_scaled_item_price_imputed|post_item_price_imputed|post_scaled_item_price_imputed|     diff_item_price|diff_scaled_item_price|
+---------------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+----------------------+-----------------------------+-----------------

### DID Purchase rate 

In [31]:
# Pre period
purchase_pre_details = spark.sql("""
with joined as (
select
    transaction_date_imputed,
    recordedTreatmentStatus,
    exposure.adUserId,
    sp.customer_id,
    receipt_id,
    transaction_id,
    item_price_imputed,
    max(date) as ads_exp_date
from exposure 
left join sp on exposure.adUserId = sp.customer_id
    -- total window method
    and transaction_date_imputed >= date_sub(exp_start_date, 14 + ads_exp_length) 
    and transaction_date_imputed < exp_start_date
group by 1,2,3,4,5,6,7
)

select 
    adUserId,
    recordedTreatmentStatus,
    case when customer_id is not null then 1 else 0 end as convert_flag
from joined
    -- only take matched ads exposure and shopper panel users
    where adUserId in (select distinct customer_id from sp_total)
group by 1,2,3
order by 1,2
""")

purchase_pre_details.show(100,False)
purchase_pre_details.createOrReplaceTempView("purchase_pre_details")

purchase_pre_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/purchase_pre_details/'
         , header=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------------------+-----------------------+------------+
|adUserId                                                            |recordedTreatmentStatus|convert_flag|
+--------------------------------------------------------------------+-----------------------+------------+
|0101000080deb73785c55df2bd61249cfbc32c96bfd2e02042ca182cdfc4eaaee31c|T                      |0           |
|0101000218b2f35b4821119af19a44577a6c7ac684796e718577e0988186b16e14a0|T                      |0           |
|01010002d7da8481d9f112676cc0631de58b515247ad33038db8f1800aeb3558db43|V3                     |0           |
|010100033caa0e6df0fc9f65807c749bb7e5b245bf3d086a4d6e5111e78472c0f38c|T                      |0           |
|010100046c1d157eab523c8a612bc990a8b497478f1cbafc4f993014cd4a64b7c949|V3                     |0           |
|010100051ae331bcf65ec335aef240c761d964c9a271141470e74bd172423b098df6|V3                     |0           |
|0101000574bbd0c60e315594ddb

In [32]:
pre_col_list = [col(col_name).alias("pre_" + col_name)  for col_name in purchase_pre_details.columns]
post_col_list = [col(col_name).alias("post_" + col_name)  for col_name in purchase_post_details.columns]
purchase_pre_details_new = purchase_pre_details.select(*pre_col_list)
purchase_pre_details_new.show()
purchase_post_details_new = purchase_post_details.select(*post_col_list)
purchase_post_details_new.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------------+----------------+
|        pre_adUserId|pre_recordedTreatmentStatus|pre_convert_flag|
+--------------------+---------------------------+----------------+
|0101000080deb7378...|                          T|               0|
|0101000218b2f35b4...|                          T|               0|
|01010002d7da8481d...|                         V3|               0|
|010100033caa0e6df...|                          T|               0|
|010100046c1d157ea...|                         V3|               0|
|010100051ae331bcf...|                         V3|               0|
|0101000574bbd0c60...|                          T|               0|
|0101000574bbd0c60...|                         V3|               0|
|010100069d9694848...|                          T|               0|
|01010007315c20751...|                          T|               0|
|010100073eb8d6f7c...|                          T|               0|
|01010007de407a216...|                          

In [33]:
# diff of pre and post
diff_purchase_details  = purchase_pre_details_new.join(purchase_post_details_new, \
                   purchase_pre_details_new.pre_adUserId == purchase_post_details_new.post_adUserId, how = "right")\
                  .withColumn("diff_convert_flag",F.col("post_convert_flag") - F.col("pre_convert_flag"))
diff_purchase_details.show()

diff_purchase_details.createOrReplaceTempView("diff_purchase_details")
diff_purchase_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/diff_purchase_details/'
         , header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------------+----------------+--------------------+----------------------------+-----------------+-----------------+
|        pre_adUserId|pre_recordedTreatmentStatus|pre_convert_flag|       post_adUserId|post_recordedTreatmentStatus|post_convert_flag|diff_convert_flag|
+--------------------+---------------------------+----------------+--------------------+----------------------------+-----------------+-----------------+
|01010002d7da8481d...|                         V3|               0|01010002d7da8481d...|                          V3|                0|                0|
|01010009895b5e00d...|                         V3|               1|01010009895b5e00d...|                          V3|                0|               -1|
|0101000b364e422f5...|                          T|               0|0101000b364e422f5...|                           T|                0|                0|
|01010017254ca6bea...|                         V3|               0|010100172

# CUPED

### CUPED Sales

In [34]:
from pyspark.mllib.stat import Statistics

def cuped_sales(df):
    df = df.withColumn("pre_item_price_imputed",col("pre_item_price_imputed").cast("float"))\
           .withColumn("post_item_price_imputed",col("post_item_price_imputed").cast("float"))\
           .withColumn("pre_scaled_item_price_imputed",col("pre_scaled_item_price_imputed").cast("float"))\
           .withColumn("post_scaled_item_price_imputed",col("post_scaled_item_price_imputed").cast("float"))
    
    var_x = df.agg({"pre_item_price_imputed": "variance"}).first()[0]
    cov_xy = df.stat.cov("pre_item_price_imputed","post_item_price_imputed")
    theta = cov_xy/var_x
    mean_x = df.agg({"pre_item_price_imputed": "mean"}).first()[0]
    print("Variance X is {}, Covariance X and Y is {}, Theta is {}, Mean of X is {}".format(var_x,cov_xy,theta,mean_x))
    
    var_x_s = df.agg({"pre_scaled_item_price_imputed": "variance"}).first()[0]
    cov_xy_s = df.stat.cov("pre_scaled_item_price_imputed","post_scaled_item_price_imputed")
    theta_s = cov_xy_s/var_x_s
    mean_x_s = df.agg({"pre_scaled_item_price_imputed": "mean"}).first()[0]
    print("Variance scaled X is {}, Covariance scaled X and scaled Y is {}, Theta scaled is {}, Mean of X is {}".format(var_x_s,cov_xy_s,theta_s, mean_x_s))
    
    df = df.withColumn("cuped_item_price", F.col("post_item_price_imputed") - theta*(F.col("pre_item_price_imputed") - mean_x_s))
    df = df.withColumn("cuped_scaled_item_price", F.col("post_scaled_item_price_imputed") - theta_s*(F.col("pre_scaled_item_price_imputed")-mean_x_s))
    return df

def cuped_pur_rate(df):
    var_x = df.agg({"pre_convert_flag": "variance"}).first()[0]
    cov_xy = df.stat.cov("pre_convert_flag","post_convert_flag")
    theta = cov_xy/var_x
    mean_x = df.agg({"pre_convert_flag": "mean"}).first()[0]
    print("Variance X is {}, Covariance X and Y is {}, Theta is {}, Mean of X is {}".format(var_x,cov_xy,theta,mean_x))
    df = df.withColumn("cuped_convert_flag", F.col("post_convert_flag") - theta*(F.col("pre_convert_flag")-mean_x))
    return df


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Sales
cuped_sales_details = cuped_sales(diff_sales_details)
cuped_sales_details.show()

cuped_sales_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/cuped_sales_details/'
         , header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-41:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 6194



Variance X is 2.71915139471753, Covariance X and Y is 0.6228424254716665, Theta is 0.22905764889798216, Mean of X is 0.027138478019216303
Variance scaled X is 141627.7616007103, Covariance scaled X and scaled Y is 34977.402018103705, Theta scaled is 0.2469671314633577, Mean of X is 6.213779720328057
+---------------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+----------------------+-----------------------------+-----------------------+------------------------------+--------------------+----------------------+------------------+-----------------------+
|pre_recordedTreatmentStatus|        pre_adUserId|     pre_customer_id|post_recordedTreatmentStatus|       post_adUserId|    post_customer_id|         customer_id|            adUserId|recordedTreatmentStatus|pre_item_price_imputed|pre_scaled_item_price_imputed|post_item_price_imputed|post_scale

In [36]:
# Purchase rate
cuped_purchase_details = cuped_pur_rate(diff_purchase_details)
cuped_purchase_details.show()

cuped_purchase_details \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/cuped_purchase_details/'
         , header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Variance X is 0.16444017717825377, Covariance X and Y is 0.027992179266207326, Theta is 0.17022712907845933, Mean of X is 0.2074923753744838
+--------------------+---------------------------+----------------+--------------------+----------------------------+-----------------+-----------------+--------------------+
|        pre_adUserId|pre_recordedTreatmentStatus|pre_convert_flag|       post_adUserId|post_recordedTreatmentStatus|post_convert_flag|diff_convert_flag|  cuped_convert_flag|
+--------------------+---------------------------+----------------+--------------------+----------------------------+-----------------+-----------------+--------------------+
|01010002d7da8481d...|                         V3|               0|01010002d7da8481d...|                          V3|                0|                0| 0.03532083136566839|
|01010009895b5e00d...|                         V3|               1|01010009895b5e00d...|                          V3|                0|               -1|-0.134

In [37]:
# bucketing
spark.sql("""select 
    post_recordedTreatmentStatus, 
    case when post_convert_flag =1 and pre_convert_flag = 0 then "new"
         when post_convert_flag =1 and pre_convert_flag = 1 then "loyal"
         when post_convert_flag =0 and pre_convert_flag = 1 then "losed"
         else "potential" end as bucket,
    sum(diff_convert_flag) as diff_convert_flag,
    count(distinct post_adUserId) as post_adUserId,
    count(distinct pre_adUserId) as pre_adUserId
from diff_purchase_details
group by 1,2
order by 2,1""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+---------+-----------------+-------------+------------+
|post_recordedTreatmentStatus|   bucket|diff_convert_flag|post_adUserId|pre_adUserId|
+----------------------------+---------+-----------------+-------------+------------+
|                           T|    losed|           -13765|        11161|       11161|
|                          V3|    losed|           -13619|        11014|       11014|
|                           T|    loyal|                0|         4525|        4525|
|                          V3|    loyal|                0|         4413|        4413|
|                           T|      new|             8682|         7099|        7099|
|                          V3|      new|             8550|         7005|        7005|
|                           T|potential|                0|        53034|       53034|
|                          V3|potential|                0|        53572|       53572|
+----------------------------+---------+--------------

# Indirect Scaling Method

In [ ]:
# attr_total = spark.sql("""
# with exposure as 
#     (select audienceFunnel, recordedTreatmentStatus, adUserId, date,
#     min(date)over() as exp_start_date, max(date)over() as exp_end_date
#     from df 
#     where audienceFunnel = '{0}'
#     group by 1,2,3,4)

    
# select
# transaction_date,
# transaction_date_imputed,
# recordedTreatmentStatus,
# customer_id,
# receipt_id,
# transaction_id,
# item_price,
# item_price_imputed,
# max(date) as ads_exp_date
# -- rank() over (partition by transaction_id order by date desc) as rk
# from exposure 
# left join sp on exposure.adUserId = sp.customer_id
# -- total window method
# and transaction_date_imputed between exp_start_date and date_add(exp_end_date, 14)

# -- remove transactions before each exposure
# and transaction_date_imputed >=date

# group by 1,2,3,4,5,6,7,8""".format(selected_audiencefunnel, brandaidids))


# attr_total2.repartition(1) \
# .write.csv("s3a://panel-ml-incrementality-data/incrementality/results_1130/"+experiment_id+"/attributed",header=True)



# Time to Convert

In [42]:
sales_post_details_first = spark.sql("""
select
    transaction_date_imputed,
    recordedTreatmentStatus,
    adUserId as customer_id,
    receipt_id,
    transaction_id,
    item_price_imputed,
    date as ads_exp_date
from exposure 
left join sp on exposure.adUserId = sp.customer_id
    -- total window method
    and transaction_date_imputed between exp_start_date and date_add(exp_end_date, 14)
    -- remove transactions before each exposure
    and transaction_date_imputed >= date
group by 1,2,3,4,5,6,7
""")

sales_post_details_first.show(100,False)
sales_post_details_first.createOrReplaceTempView("sales_post_details_first")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-----------------------+--------------------------------------------------------------------+----------+--------------+------------------+------------+
|transaction_date_imputed|recordedTreatmentStatus|customer_id                                                         |receipt_id|transaction_id|item_price_imputed|ads_exp_date|
+------------------------+-----------------------+--------------------------------------------------------------------+----------+--------------+------------------+------------+
|null                    |T                      |010100005ea44879a5fe5fab4f4199133ca5cd31aa7c515bf4dad134dd9b742ca8ec|null      |null          |null              |2022-10-25  |
|null                    |T                      |0101000061c5b8e01a8dd73a69832f91fd1ec0e84a6a0e642dfb3940c2d1c76e6e6d|null      |null          |null              |2022-08-17  |
|null                    |T                      |0101000068ff60d325d1cb94d81dca693f9b829c6a01eacc21214f2853ee

In [43]:
# If multiple ads exposure to one customer, take first ads exp date, compare to transaction date
first_ads_exp_attr = spark.sql("""
             select * from (
                 select recordedTreatmentStatus, customer_id, transaction_date_imputed, min(ads_exp_date) as closest_ads_exp_date,
                     datediff(transaction_date_imputed, min(ads_exp_date)) as days_convert
                 from sales_post_details_first
                 where customer_id is not null
                       -- and transaction_date_imputed>= ads_exp_date
                 group by 1,2,3
                 order by 1,2,3)
             where days_convert>=0 """)

first_ads_exp_attr.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+--------------------------------------------------------------------+------------------------+--------------------+------------+
|recordedTreatmentStatus|customer_id                                                         |transaction_date_imputed|closest_ads_exp_date|days_convert|
+-----------------------+--------------------------------------------------------------------+------------------------+--------------------+------------+
|T                      |0101000574bbd0c60e315594ddbe741ccfcbc90f0cb866d8465438fa03dc8d7481a0|2022-10-01              |2022-08-29          |33          |
|T                      |0101000a25a596b43c95d75d45e051601cb45322d7de646e521b4bdda9f1616c22d9|2022-10-30              |2022-08-20          |71          |
|T                      |0101000edf0e75ab87ef4723bb60dfe400a6ef655de4a8678d548b37a475114c8e37|2022-11-06              |2022-08-21          |77          |
|T                      |0101000f3328411e6493c4c96b99f39d9b7da1ddd6cbd0a6b81

In [44]:
first_ads_exp_attr.groupby(F.col("recordedTreatmentStatus")).agg(F.avg("days_convert")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------+
|recordedTreatmentStatus|avg(days_convert)|
+-----------------------+-----------------+
|                      T|38.86746271882429|
|                     V3|39.18612568367005|
+-----------------------+-----------------+

In [45]:
first_ads_exp_attr \
    .repartition(1) \
    .write.mode('overwrite') \
    .csv('s3a://panel-ml-incrementality-data/incrementality/results/results_'+experiment+'/'+today+'/first_ads_exp_attr/'
         , header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…